# 발표 소단원 리스트

(4, 14)
(5, 2)
(5, 6)
(5, 10)
(5, 14)
(5, 18)
(6, 1)
(6, 5)

# 전체 요약자료

(4, 14) : [flattening nested sequence] : yield from

(5, 2) : [Printing to a File] : text모드로 파일 연 뒤, print(내용,파일)

(5, 6) : [file like object I/O] : io.StringIO() ,io.BytesIO()

(5, 10) : [

(5, 14) :

(5, 18) :

(6, 1) :

(6, 5) :

### 4-14) Flattening a Nested Sequence
- (sol) yield from + recursive algorithm 사용
- yield from은 emit all of its values!!


In [4]:
from collections import Iterable # iterable 객체 테스트
def flatten(items,ignore_types = (str,bytes)):
    for x in items:
        if isinstance(x,Iterable) and not isinstance(x,ignore_types):
            yield from flatten(x)
#            for i in flatten(x):
#                print(i)
        else:
            yield x

In [5]:
items = [1,2,[3,4,[5,6],7],8]

In [6]:
for x in flatten(items):
    print(x)

1
2
3
4
5
6
7
8


In [9]:
isinstance('abc',Iterable)

True

In [10]:
isinstance(b'12ab',Iterable)

True

### yield from은 다른 제너레이터를 서브루틴에 불러올 때 굉장히 유용하다!!

### 나중에 corountines 나 generator-based concurrency 다룰 때 (12.12) 다시 등장!!

### 5-2) Printing to a File
- (sol) text모드로 연 뒤, print(내용,파일)

In [14]:
## 반드시 text모드로 열어야함!!
with open('somefile.txt','wt') as f:
    print('Hello World!',file=f)

### 5-6) Performing I/O Operations on a String
- 파일과 같은 느낌의 객체에 read,write a text or binary string
- (sol) io.StringIO() ,io.BytesIO()

- unit test에 사용됨 하지만 file descriptor가 없어서 실제 파일과 완전히 동일하게 작동하지는 않음

In [17]:
import io
s = io.StringIO()
s.write('Hello World\n')

12

In [18]:
print('This is a test',file=s)
s.getvalue()

'Hello World\nThis is a test\n'

In [21]:
### wrap on s
s = io.StringIO('Hello\nWorld\n')
s.read(5)

'Hello'

In [22]:
s.read()

'\nWorld\n'

In [23]:
## binary string
s = io.BytesIO()
s.write(b'binary data')
s.getvalue()

b'binary data'

### 5-10) Memory Mapping Binary Files
- 바이너리 파일을 메모리에 매핑시켜서 slice나 access / in-place modification 하고자 함
- (sol) mmap

### 추가
- seek(),read(),write() 안 쓰고, slice방식으로 파일 내용 접근 및 수정 가능!!
- 매핑시 다 메모리에 올리는게 아니라, 예상 공간 만들고 불릴 때마다 올리기!!(효율적)
- 동시에 여러 사람이 동일한 파일에 읽기/수정도 가능

In [28]:
import os
import mmap
def memory_map(filename,access=mmap.ACCESS_WRITE):
    size = os.path.getsize(filename)
    fd = os.open(filename,os.O_RDWR)
    return mmap.mmap(fd,size,access=access) # WRITE로 하면 R/W 다 됨..

In [34]:
size = 100000
# binary로 열기
with open('data','wb') as f:
    f.seek(size-1) 
    f.write(b'\x00')

In [35]:
## 메모리 매핑 방식
m = memory_map('data')
len(m)

100000

In [36]:
m[0:10]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [37]:
m[0:11] = b'Hello World'
m.close() # 반드시 close 필요!

In [38]:
with open('data','rb') as f:
    print(f.read(11))

b'Hello World'


In [39]:
with memory_map('data') as m:
    print(len(m))
    print(m[0:11])

100000
b'Hello World'


In [40]:
m.closed

True

In [ ]:
### Read only
m = memory_map(filename,mmap.ACCESS_READ)
## locally하게 m만 바뀌고, file은 변함 없게
m = memory_map(filename,mmap.ACCESS_COPY)

In [ ]:
## seek(),read(),write() 안 쓰고, slice방식으로 파일 내용 접근 및 수정 가능!!

### 5-14) Bypassing Filename Encoding
- 인코딩이 이상한 파일들을 불러오기
- (sol) raw한 bytes string 값으로 직접 쓰기!! (디코딩이 잘못될 우려를 방지!!)

In [41]:
with open('jalape\xf1o.txt','w') as f:
    f.write('Spicy!')

In [43]:
import os

[f for f in os.listdir() if f.endswith('txt')]

['somefile.txt', 'jalapeño.txt']

In [44]:
os.listdir(b'.') # undecoded bytes string

[b'somefile.txt',
 b'[3\xec\xa3\xbc\xec\xb0\xa8] \xec\xa0\x95\xec\xa7\x80\xed\x95\x9c.ipynb',
 b'README.md',
 b'jalape\xc3\xb1o.txt',
 b'\xec\xa0\x84\xec\xb2\xb4 \xea\xb3\x84\xed\x9a\x8d\xed\x91\x9c.ipynb',
 b'.ipynb_checkpoints',
 b'data',
 b'2\xec\xa3\xbc\xec\xb0\xa8',
 b'1\xec\xa3\xbc\xec\xb0\xa8',
 b'\xec\x8a\xa4\xed\x84\xb0\xeb\x94\x94_\xeb\xb0\x9c\xed\x91\x9c\xed\x83\xac\xed\x94\x8c\xeb\xa6\xbf.ipynb']

In [45]:
with open(b'jalape\xc3\xb1o.txt') as f:
    print(f.read())

Spicy!


### 5-18) Wrapping an Existing File Descriptor as a file object
- (sol) low_level file descriptor를 파일이름 쓰는 자리에 넣기

In [ ]:
import os
## open a low-level file descriptor
fd = os.open('somefile.txt',os.O_WRONLY | os.O_CREAT)

## turn into a proper file
f = open(fd,'wt')
f.write('hello world\n')
f.close()

## f.close()하면 file descriptor도 자동으로 꺼지는데 이를 방지
#f = open(fd,'wt',closefd=False)

In [ ]:
## client socket 받아서 내용씀!!
## 파일로 만들어주는 makefile() 방법보다 더 효율적!!
def echo_client(client_sock, addr):
    print("Got connection from", addr)

    # Make text-mode file wrappers for socket reading/writing
    client_in = open(client_sock.fileno(), 'rt', encoding='latin-1', closefd=False)
    client_out = open(client_sock.fileno(), 'wt', encoding='latin-1', closefd=False)

    # Echo lines back to the client using file I/O
    for line in client_in:
        client_out.write(line)
        client_out.flush()
    client_sock.close()
    
def echo_server(address):
    sock = socket(AF_INET, SOCK_STREAM)
    sock.bind(address)
    sock.listen(1)
    while True:
        client, addr = sock.accept()
        echo_client(client, addr)

In [ ]:
## 이미 open된 file을 다른방식으로 이용할 때도 유용
import sys
## Create a binary-mode file for stdout
bstdout = open(sys.stdout.fileno(),'wb',closefd=False)
bstdout.write(b'Hello World\n')
bstdout.flush()

### 6-1) Reading and Writing CSV data
- (sol) import csv

- 실제로는 pd.read_csv 사용..

In [ ]:
import csv

'''
Symbol,Price,Date,Time,Change,Volume
"AA",39.48,"6/11/2007","9:36am",-0.18,181800
"AIG",71.38,"6/11/2007","9:36am",-0.15,195500
"AXP",62.58,"6/11/2007","9:36am",-0.46,935000
"BA",98.31,"6/11/2007","9:36am",+0.12,104800
"C",53.08,"6/11/2007","9:36am",-0.25,360900
"CAT",78.29,"6/11/2007","9:36am",-0.23,225400
'''

## 기본은 tuple로 읽음
print('Reading as tuples:')
with open('stocks.csv') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    for row in f_csv:
        print('    ',row)

## namedtuples로 바꾸기

print('Reading as namedtuples')
from collections import namedtuple
with open('stocks.csv') as f:
    f_csv = csv.reader(f)
    Row = namedtuple('Row', next(f_csv))
    for r in f_csv:
        row = Row(*r)
        # Process row
        print('    ', row)


print('Reading as dicts')
with open('stocks.csv') as f:
    f_csv = csv.DictReader(f)
    for row in f_csv:
        # process row
        print('    ', row)

# 타입 바꾸기

col_types = [str, float, str, str, float, int]
with open('stocks.csv') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    for row in f_csv:
        # Apply conversions to the row items
        row = tuple(convert(value) for convert, value in zip(col_types, row))
        print(row)

print('Reading as dicts with type conversion')

field_types = [ ('Price', float),
                ('Change', float),
                ('Volume', int) ]

with open('stocks.csv') as f:
    for row in csv.DictReader(f):
        row.update((key, conversion(row[key])) 
                   for key, conversion in field_types)
        print(row)

### 6-5) Turning a Dictionary into XML
- dict을 xml로 바꾸기
- (sol) from xml.etree.ElementTree import Element 사용

In [46]:
from xml.etree.ElementTree import Element

def dict_to_xml(tag,d):
    '''
    Turn a simple dict of key/value pairs into xml
    '''
    
    elem = Element(tag)
    
    for key,val in d.items():
        child = Element(key)
        child.text = str(val)
        
        elem.append(child)
    return elem

In [47]:
s = {'name':'GOOG','shares':100,'price':490.1}
e = dict_to_xml('stock',s)
e

<Element 'stock' at 0x1068c09a8>

In [48]:
## I/O 를 위해 bytes string으로 바꾸기 
from xml.etree.ElementTree import tostring
tostring(e)

b'<stock><name>GOOG</name><shares>100</shares><price>490.1</price></stock>'

In [49]:
### attach attributes!!
e.set('_id','1234')
tostring(e)

b'<stock _id="1234"><name>GOOG</name><shares>100</shares><price>490.1</price></stock>'

In [50]:
## 비추하는 방법
# 1. escape 처리 제대로 못함
# 2. 큰 xml 문서 만들기 복잡해짐..
def dict_to_xml_str(tag,d):
    parts = ['<{}>'.format(tag)]
    for key,val in d.items():
        parts.append('<{0}>{1}</{0}>'.format(key,val))
    parts.append('</{}>'.format(tag))
    return ''.join(parts)

In [52]:
d = {'name':'<spam>'}
dict_to_xml_str('item',d)

'<item><name><spam></name></item>'

In [53]:
e=dict_to_xml('item',d)
tostring(e)

b'<item><name>&lt;spam&gt;</name></item>'

In [54]:
from xml.sax.saxutils import escape,unescape
escape('<spam>')

'&lt;spam&gt;'

In [55]:
unescape(_)

'<spam>'